In [1]:
import os
import SimpleITK
import pydicom
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize

ImportError: cannot import name '_validate_lengths'

In [3]:
#将所有mask出现过的地方置白
xx = cv2.imread('D:/work/pytest/Taidi/unet_test/dataset_ori/1001/arterial phase/100110001_mask.png',0)
xx[xx<=256.0] = 0

labelpath = 'D:/work/pytest/Taidi/New_unet/datasetlabel'
last = xx
for file in os.listdir(labelpath):
    yy = cv2.imread(labelpath + '/'+ file,0)
    last = np.add(yy,last)
last = np.where(last<0.5,0.0,255.0) 
cv2.imwrite('D:/work/pytest/Taidi/unet_test/'+ 'yy'+'.png',last)

True

In [4]:
#将同一的直肠大概位置mask膨胀腐蚀一下
mask = cv2.imread('D:/work/pytest/Taidi/New_unet/'+ 'yy.png',0)
eroded = morphology.erosion(mask, np.ones([4, 4]))
dilation = morphology.dilation(eroded, np.ones([15, 15]))
cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'yy1'+'.png',dilation)

True

In [7]:
#测试原图加上结节mask
img = cv2.imread('D:/work/pytest/Taidi/New_unet/datasetimage/101820003.png',0)
mask = cv2.imread('D:/work/pytest/Taidi/New_unet/yy1.png',0)
img = img * (mask/255.0)
# renormalizing the masked image (in the mask region)
#
new_mean = np.mean(img[mask > 0])
new_std = np.std(img[mask > 0])
        #
        #  Pulling the background color up to the lower end
        #  of the pixel range for the lungs
        #
old_min = np.min(img)  # background color
if new_mean - 6.0 * new_std > 0.0:
    img[img == old_min] = new_mean - 6.0 * new_std  # resetting backgound color
#img = img - new_mean
#img = img / new_std
cv2.imwrite('D:/work/pytest/Taidi//New_unet/'+ 'imgmask1'+'.png',img)


True

In [8]:
print(new_mean,new_std)

182.5782241497375 13.989049444335265


In [72]:
#min_row,max_row,min_col,max_col = 220,440,145,365

In [73]:
#img = img[min_row:max_row, min_col:max_col]
#cv2.imwrite('D:/work/pytest/Taidi//New_unet/'+ 'imgmaskcut'+'.png',img)

True

In [9]:
#测试原图加上结节mask(2)
#img = img - new_mean
#img = img / new_std

        # make image bounding box  (min row, min col, max row, max col)
labels = measure.label(mask)
regions = measure.regionprops(labels)
        #
        # Finding the global min and max row over all regions
        #
min_row = 512
max_row = 0
min_col = 512
max_col = 0
for prop in regions:
    B = prop.bbox
    if min_row > B[0]:
        min_row = B[0]
    if min_col > B[1]:
        min_col = B[1]
    if max_row < B[2]:
        max_row = B[2]
    if max_col < B[3]:
        max_col = B[3]
width = max_col - min_col
height = max_row - min_row
if width > height:
    max_row = min_row + width
else:
    max_col = min_col + height
        #
        # cropping the image down to the bounding box for all regions
        # (there's probably an skimage command that can do this in one line)
        #
img = img[min_row:max_row, min_col:max_col]
mask = mask[min_row:max_row, min_col:max_col]
cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'imgimg'+'.png',img)
cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'imgmask2'+'.png',mask)

True

In [15]:
node_mask = cv2.imread('D:/work/pytest/Taidi/New_unet/datasetlabel/101820003_mask.png',0)#!
if max_row - min_row < 5 or max_col - min_col < 5:  # skipping all images with no god regions
    pass
else:
            # moving range to -1 to 1 to accomodate the resize function
    mean = np.mean(img)
    img = img - mean
    mini = np.min(img)
    maxi = np.max(img)
    img = img / (maxi - mini)
    new_img = resize(img, [512, 512])
    new_node_mask = resize(node_mask[min_row:max_row, min_col:max_col], [512, 512])
    #out_images.append(new_img)
    #out_nodemasks.append(new_node_mask)
    cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'newimgimg'+'.png',new_img*255)
    cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'newimgmask2'+'.png',new_node_mask*255)

In [18]:
print(maxi,mini)

0.7383647605938479 -0.26163523940615196


In [12]:
new_node_mask.shape

(512, 512)

In [13]:
np.max(new_node_mask)

1.0

测试部分结束，开始正式批量操作部分

In [3]:
from glob import glob
working_path = "D:/work/pytest/Taidi/New_unet/datasetimg"
file_list = glob(working_path + '/'+"*.png")#glob文件名匹配

In [4]:
file_list.sort()

In [5]:
print(file_list)

['D:/work/pytest/Taidi/New_unet/datasetimg\\101720013.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720014.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720015.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720016.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720017.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720018.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720019.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720020.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720021.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720022.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101720023.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101820001.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101820002.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101820003.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101820004.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101820005.png', 'D:/work/pytest/Taidi/New_unet/datasetimg\\101820006.pn

In [10]:
ee,ff = file_list[0].split('\\')
ff

'101720013.png'

In [16]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from glob import glob

working_path = "D:/work/pytest/Taidi/New_unet/datasetimg/"
file_list = glob(working_path + "*.png")#glob文件名匹配
file_list.sort()

out_images = []  # final set of images
out_nodemasks = []  # final set of nodemasks
maskpub = cv2.imread('D:/work/pytest/Taidi/New_unet/yy1.png',0)
for fname in file_list:
    print("working on file ", fname)
    img = cv2.imread(fname,0)
    ee,ff = fname.split('\\')
    node_mask = cv2.imread('D:/work/pytest/Taidi/New_unet/datasetlab'+'/'+ff,0)
    mask = maskpub
    new_size = [512, 512]  # we're scaling back up to the original size of the image
    img = img * (mask/255.0)  # apply public roi mask   !!!!!!
        #
        # renormalizing the masked image (in the mask region)
        #
    new_mean = np.mean(img[mask > 0])
    new_std = np.std(img[mask > 0])
        #
        #  Pulling the background color up to the lower end
        #  of the pixel range for the lungs
        #
    old_min = np.min(img)  # background color
    img[img == old_min] = new_mean - 6.0 * new_std  # resetting backgound color
        #img = img - new_mean
        #img = img / new_std
        # make image bounding box  (min row, min col, max row, max col)
    labels = measure.label(mask)
    regions = measure.regionprops(labels)
        #
        # Finding the global min and max row over all regions
        #
    min_row = 512
    max_row = 0
    min_col = 512
    max_col = 0
    for prop in regions:
        B = prop.bbox
        if min_row > B[0]:
            min_row = B[0]
        if min_col > B[1]:
            min_col = B[1]
        if max_row < B[2]:
            max_row = B[2]
        if max_col < B[3]:
            max_col = B[3]
    width = max_col - min_col
    height = max_row - min_row
    if width > height:
        max_row = min_row + width
    else:
        max_col = min_col + height
        #
        # cropping the image down to the bounding box for all regions
        # (there's probably an skimage command that can do this in one line)
        #
    img = img[min_row:max_row, min_col:max_col]
    mask = mask[min_row:max_row, min_col:max_col]
    if max_row - min_row < 5 or max_col - min_col < 5:  # skipping all images with no god regions
        pass
    else:
            # moving range to -1 to 1 to accomodate the resize function
        mean = np.mean(img)
        img = img - mean
        mini = np.min(img)
        maxi = np.max(img)
        img = img / (maxi - mini)
        new_img = resize(img, [512, 512])
        new_node_mask = resize(node_mask[min_row:max_row, min_col:max_col], [512, 512])
        out_images.append(new_img)
        out_nodemasks.append(new_node_mask)

num_images = len(out_images)
#
#  Writing out images and masks as 1 channel arrays for input into network
#
final_images = np.ndarray([num_images, 1, 512, 512], dtype=np.float32)
final_masks = np.ndarray([num_images, 1, 512, 512], dtype=np.float32)
for i in range(num_images):
    final_images[i, 0] = out_images[i]
    final_masks[i, 0] = out_nodemasks[i]


rand_i = np.random.choice(range(num_images), size=num_images, replace=False)
test_i = int(0.2 * num_images)
savepath = 'D:/work/pytest/Taidi/New_unet/npy/'
np.save(savepath + "trainImages.npy", final_images[rand_i[test_i:]])
np.save(savepath + "trainMasks.npy", final_masks[rand_i[test_i:]])
np.save(savepath + "testImages.npy", final_images[rand_i[:test_i]])
np.save(savepath + "testMasks.npy", final_masks[rand_i[:test_i]])

working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720013.png


D:\work\python\anaconda1\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
D:\work\python\anaconda1\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720014.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720015.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720016.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720017.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720018.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720019.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720020.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720021.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720022.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101720023.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101820001.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101820002.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101820003.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\101820

working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103320015.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103320016.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103320017.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103320018.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103320019.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103320020.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420008.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420009.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420010.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420011.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420012.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420013.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420014.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\103420

working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220016.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220017.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220018.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220019.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220020.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220021.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220022.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220023.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220024.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220025.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220026.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105220027.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105320002.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\105320

working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920008.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920009.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920010.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920011.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920012.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920013.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920014.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920015.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\106920016.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\107020008.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\107020009.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\107020010.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\107020011.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\107020

working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108720021.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108720022.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108720023.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108720024.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820014.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820015.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820016.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820017.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820018.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820019.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820020.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820021.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820022.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\108820

working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620016.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620017.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620018.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620019.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620020.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620021.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110620022.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720004.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720005.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720006.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720007.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720008.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720009.png
working on file  D:/work/pytest/Taidi/New_unet/datasetimg\110720

In [20]:
cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'testimg101820005'+'.png',out_images[15]*255)
cv2.imwrite('D:/work/pytest/Taidi/New_unet/'+ 'testmask101820005'+'.png',out_nodemasks[15]*255)

True

In [18]:
out_nodemasks[3]

(512, 512)

In [11]:
rand_ii = np.random.choice(range(1172), size=1172, replace=False)
rand_ii

array([ 624,  334,  227, ..., 1127,  261,  966])

In [13]:
test_ii = int(0.2 * 1172)
test_ii

234

最后一步：进行训练和预测

In [1]:
from __future__ import print_function

import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

working_path = 'D:/work/pytest/Taidi/New_unet/npy/'

K.set_image_dim_ordering('th')  # Theano dimension ordering in this code

img_rows = 512
img_cols = 512

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_np(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((1, img_rows, img_cols))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    model.compile(optimizer=Adam(lr=1.0e-5), loss=dice_coef_loss, metrics=[dice_coef])#metrices评价函数和 损失函数 相似，只不过评价函数的结果不会用于训练过程中。

    return model

D:\work\python\anaconda1\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
savepath = 'D:/work/pytest/Taidi/New_unet/npy/'

In [13]:
def train_and_predict(use_existing):
    print('-' * 30)
    print('Loading and preprocessing train data...')
    print('-' * 30)
    imgs_train = np.load(working_path + "trainImages.npy").astype(np.float32)
    imgs_mask_train = np.load(working_path + "trainMasks.npy").astype(np.float32)

    imgs_test = np.load(working_path + "testImages.npy").astype(np.float32)
    imgs_mask_test_true = np.load(working_path + "testMasks.npy").astype(np.float32)

    #mean = np.mean(imgs_train)  # mean for data centering
    #std = np.std(imgs_train)  # std for data normalization

    #imgs_train -= mean  # images should already be standardized, but just in case
    #imgs_train /= std

    print('-' * 30)
    print('Creating and compiling model...')
    print('-' * 30)
    # model = get_unet()
    model = get_unet()
    # Saving weights to unet.hdf5 at checkpoints
    #model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', save_best_only=True)
    #
    # Should we load existing weights?
    # Set argument for call to train_and_predict to true at end of script
    
    #model.load_weights(working_path + 'unet.hdf5')

    #
    # The final results for this tutorial were produced using a multi-GPU
    # machine using TitanX's.
    # For a home GPU computation benchmark, on my home set up with a GTX970
    # I was able to run 20 epochs with a training set size of 320 and
    # batch size of 2 in about an hour. I started getting reseasonable masks
    # after about 3 hours of training.
    #
    #print('-' * 30)
    #print('Fitting model...')
    #print('-' * 30)
    #model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=80, verbose=1, shuffle=True,
    #          callbacks=[model_checkpoint])

    # loading best weights from training session
    print('-' * 30)
    print('Loading saved weights...')
    print('-' * 30)
    model.load_weights(working_path + 'unet.hdf5')

    print('-' * 30)
    print('Predicting masks on test data...')
    print('-' * 30)
    num_test = len(imgs_test)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512], dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([imgs_test[i:i + 1]], verbose=1)[0]
    np.save(savepath + 'masksTestPredicted.npy', imgs_mask_test)    

if __name__ == '__main__':
    train_and_predict(True)


------------------------------
Loading and preprocessing train data...
------------------------------
------------------------------
Creating and compiling model...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 5s 5s/step


In [5]:
model = get_unet()
model.load_weights(working_path + 'unet.hdf5')

In [ ]:
imgs_mask_test_true = np.load(working_path + "testMasks.npy").astype(np.float32)
imgs_test = np.load(working_path + "testImages.npy").astype(np.float32)
num_test = len(imgs_test)
imgs_mask_test = np.ndarray([num_test, 1, 512, 512], dtype=np.float32)
mean = 0.0
for i in range(num_test):
    mean += dice_coef_np(imgs_mask_test_true[i, 0], imgs_mask_test[i, 0])
mean /= num_test
print("Mean Dice Coeff : ", mean)

score = model.evaluate(imgs_mask_test,imgs_mask_test_true, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Mean Dice Coeff :  0.2736196911070229


In [9]:
#目前focal0.2_0.75效果最好
working_path = 'D:/work/pytest/Taidi/New_unet/ToService/focal_0.2_0.75/'
savepath = 'D:/work/pytest/Taidi/New_unet/npy/'
masktestpred = np.load(working_path + "masksTestPredicted.npy").astype(np.float32)
imgs_mask_test_true = np.load(savepath + "testMasks.npy").astype(np.float32)

In [10]:
for k in range(30):
    slice_npy = masktestpred[k,0,:,:]
    slice_npy2 = imgs_mask_test_true[k,0,:,:]
    slice_npy = np.where(slice_npy > np.mean(slice_npy),1.,0.)
    slice_npy2 = np.where(slice_npy2 > np.mean(slice_npy2),1.,0.)
    cv2.imwrite('D:/work/pytest/Taidi/New_unet/final/'+ str(k+60) +'.png',slice_npy*255)
    cv2.imwrite('D:/work/pytest/Taidi/New_unet/final/'+ str(k+60) + '_true'+'.png',slice_npy2*255)
#    print(np.max(masktestpred[k,0,:,:]*255*255*255))

In [24]:
#np.mean(thr)
np.min(slice_npy)

0.0

In [ ]:
for k in range(20):
    thr = np.where(slice_npy > np.mean(slice_npy),0.,1.0)

In [ ]:
#focal损失
from keras import backend as K
'''
Compatible with tensorflow backend
'''
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [ ]:
#tversky损失
def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)
def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)
